# Introduction

### Pydantic Basics: Creating and Using Models

Pydantic modules are the foundation of data validataion in Python. They use python type annotations to define the structure and validation data at runtime. Here's a detailed exploration of basic model creation with several examples.

In [1]:
from pydantic import BaseModel 



In [2]:
class Person(BaseModel):
    name: str
    age: int
    city: str

person = Person(name="Boon sai", age=35, city="Tirupati")

print(person)

name='Boon sai' age=35 city='Tirupati'


In [3]:
type(person)

__main__.Person

# Difference b/w pydantic and data classes

differences between pydantic base class and data class

In [4]:
from dataclasses import dataclass

@dataclass
class Person():
    name: str
    age: int
    city: str
person = Person(name="Boon sai", age=35, city="Tirupati")

print(person)

Person(name='Boon sai', age=35, city='Tirupati')


here the main difference between the data class and base model is-

pydantic does data validation , for every invalid input it returns error 

here is the example

In [12]:
# data classes
@dataclass
class Person:
    name: str
    age: int
    city: str


# pydantic class

class Person1(BaseModel):
    name: str
    age: int
    city: str


person = Person(name="boon sai", age=23, city="Tirupati")
print(person)

person2 = Person(name="boon sai", age=23, city=2378934) # it wont give you error
print(person2)
print('----------- \n A pydantic output')
person1 = Person1(name="boon sai", age = '23', city="Tirupati") # Pydantic can able to typecast from str -> int / float if it is a number , but not int/float -> str
print(person1)

person2 = Person1(name="boon sai", age=23, city=2378934) # here from number -> str it wont convert it gives you error
print(person2) 

Person(name='boon sai', age=23, city='Tirupati')
Person(name='boon sai', age=23, city=2378934)
----------- 
 A pydantic output
name='boon sai' age=23 city='Tirupati'


ValidationError: 1 validation error for Person1
city
  Input should be a valid string [type=string_type, input_value=2378934, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type

# Model with optional fields
Add optional fields using Python's optional type:

In [13]:
from typing import Optional

class Employee(BaseModel):
    id: int
    name: str
    department: str
    salary: Optional[float] = None # Optional with default value None
    is_active: Optional[bool] = True # Optional with default value True


In [15]:
# Examples with and without optional fields
emp1 = Employee(id=1, name="John", department="IT")
print(emp1) # id=1 name='John' department='IT' salary=None is_active=True

id=1 name='John' department='IT' salary=None is_active=True


In [16]:
emp2 = Employee(id=2, name="john", department='IT', salary=60000, is_active=False)
print(emp2) # here salary does automatic type-casting float -> int

id=2 name='john' department='IT' salary=60000.0 is_active=False


Definition:
- Optional[type]: indicates the field can be None
- Default value(= None or = True): Makes the field optional
- Required fields must stll be provided
- Pydantic validates types even for optional fields when values are provided

In [28]:
from pydantic import BaseModel
from typing import List

class Classroom(BaseModel):
    room_number: str
    students: list[str] # list of strings
    capacity: int

In [29]:
classroom  = Classroom(
    room_number="A101",
    students=['ALice', 'Bob', 'Charlie'],
    capacity=30
)
print(classroom)

room_number='A101' students=['ALice', 'Bob', 'Charlie'] capacity=30


it can type cast from list to tuple / sets , but not like dictionary

type castings possible

- list -> tuple
- list -> set
- set -> list
- set -> tuple

but not 

- tuple -> list
- tuple -> set

In [30]:
classroom  = Classroom(
    room_number="A101",
    students={'ALice', 'Bob', 'Charlie'},
    capacity=30
)
print(classroom)

classroom  = Classroom(
    room_number="A101",
    students=('ALice', 'Bob', 'Charlie'),
    capacity=30
)
print(classroom)

room_number='A101' students=['Charlie', 'Bob', 'ALice'] capacity=30
room_number='A101' students=['ALice', 'Bob', 'Charlie'] capacity=30


In [32]:
try:
    invalid_val = Classroom(room_number='A1', students=['ram', 123], capacity=30)

except ValueError as e:
    print(e)

# here pydantic raises error

1 validation error for Classroom
students.1
  Input should be a valid string [type=string_type, input_value=123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type


# Models with Nested Models

Create complex structures with nested models

In [37]:
from pydantic import BaseModel

class Address(BaseModel):
    street: str
    city: str
    zip_code: int

class Customer(BaseModel):
    customer_id: int
    name: str
    address: Address # Nested Model

customer = Customer(
    customer_id=1,
    name="Emma",
    address={"street": '123 Main st', 'city': 'Boston', 'zip_code': '01234'} # here all the parameters are taken in terms of dictionary and even it is converting the str -> int

)

print(customer)

customer_id=1 name='Emma' address=Address(street='123 Main st', city='Boston', zip_code=1234)


# Pydantic Fields: Customization and Constraints

The Field function in pydantic enhances model fields beyond basic type hints by allowing you to specify validation rules, default values, aliases, and more. Here's comprehensive tutorial with examples.

In [41]:
from pydantic import BaseModel, Field

class Item(BaseModel):
    name: str = Field(min_length=2, max_length=50)
    price: float = Field(gt=0, le=100) # gt= greater than, le= less that
    quantity: int = Field(ge=0) # ge = greater than or equal to

item = Item(name="book", price=10, quantity=10)
print(item)

item = Item(name="book", price=-1, quantity=10) # it gives error
print(item)

name='book' price=10.0 quantity=10


ValidationError: 1 validation error for Item
price
  Input should be greater than 0 [type=greater_than, input_value=-1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/greater_than

In [42]:
class User(BaseModel):
    user_name : str = Field(..., description="Unique username for the user")
    age: int = Field(default=18, description="User age, defaults to 18")
    email: str = Field(default_factory=lambda: "user@example.com", description= "Default email address")

# Examples
user1 = User(user_name='alice')
print(user1)

user2 = User(user_name='bob', age=25, email='bob@domain.com')
print(user2)

user_name='alice' age=18 email='user@example.com'
user_name='bob' age=25 email='bob@domain.com'


# you can even get schema in terms of json

it makes the developer to understand what kind of request is sending

In [44]:
print(User.model_json_schema())

{'properties': {'user_name': {'description': 'Unique username for the user', 'title': 'User Name', 'type': 'string'}, 'age': {'default': 18, 'description': 'User age, defaults to 18', 'title': 'Age', 'type': 'integer'}, 'email': {'description': 'Default email address', 'title': 'Email', 'type': 'string'}}, 'required': ['user_name'], 'title': 'User', 'type': 'object'}
